In [1]:
from sklearn.metrics import log_loss,accuracy_score,roc_auc_score
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import argparse
import os
from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import pickle
import sys
import xgboost
from xgboost import XGBRegressor 
from sklearn.model_selection import train_test_split
import time
import copy
from google.colab import drive

drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/ML/Flight price predictor')
!ls

Mounted at /content/drive
 Data_Train.xlsx		 Sample_submission.xlsx
'Flight price predictor.ipynb'	 Test_set.xlsx


In [2]:
df_train = pd.read_excel('Data_Train.xlsx',header=0,parse_dates=[1,4])
df_test=pd.read_excel('Test_set.xlsx',header=0,parse_dates=[1,4])
train_orig=df_train.copy(deep=True)
test_orig=df_test.copy(deep=True)

In [3]:
df_train.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,2019-03-24,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,2019-01-05,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,2019-09-06,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,2019-12-05,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,2019-01-03,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [4]:
df_train['flight_month'] = df_train['Date_of_Journey'].dt.month
df_test['flight_month'] = df_test['Date_of_Journey'].dt.month
df_train['WEEKEND'] = ((pd.DatetimeIndex(df_train['Date_of_Journey']).dayofweek)//5+1).astype(int)
df_test['WEEKEND'] = ((pd.DatetimeIndex(df_test['Date_of_Journey']).dayofweek)//5+1).astype(int)
df_train['FLIGHTWEEK'] = df_train['Date_of_Journey'].dt.week
df_test['FLIGHTWEEK'] = df_test['Date_of_Journey'].dt.week
df_train['WEEKDAY'] = df_train['Date_of_Journey'].dt.dayofweek
df_test['WEEKDAY'] = df_train['Date_of_Journey'].dt.dayofweek

In [42]:
df_train.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,flight_month,WEEKEND,FLIGHTWEEK,WEEKDAY,Flight hours
0,0,2019-03-24,5,3,BLR → DEL,22:20,01:10 22 Mar,2h 50m,0,0,3897,3,2,12,6,2
1,2,2019-01-05,2,5,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2,0,7662,1,2,1,5,7
2,1,2019-09-06,3,6,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2,0,13882,9,1,36,4,19
3,0,2019-12-05,2,5,CCU → NAG → BLR,18:05,23:30,5h 25m,1,0,6218,12,1,49,3,5
4,0,2019-01-03,5,3,BLR → NAG → DEL,16:50,21:35,4h 45m,1,0,13302,1,1,1,3,4


In [43]:
df_test.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,flight_month,WEEKEND,FLIGHTWEEK,WEEKDAY,Flight hours
0,1,2019-06-06,3,6,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1,0,6,1,23,6,10
1,0,2019-12-05,2,5,CCU → MAA → BLR,06:20,10:20,4h,1,0,12,1,49,5,4
2,1,2019-05-21,3,6,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1,1,5,1,21,4,23
3,4,2019-05-21,3,6,DEL → BOM → COK,08:00,21:00,13h,1,0,5,1,21,3,13
4,7,2019-06-24,5,3,BLR → DEL,23:55,02:45 25 Jun,2h 50m,0,0,6,1,26,3,2


In [7]:
dict_cities = {'Mumbai':1 , 'Kolkata' : 2, 'Delhi' : 3, 'Chennai' : 4, 'Banglore': 5,'New Delhi':3,'Cochin':6,'Hyderabad':7}
#df_train = df_train.replace({'Source':dict_cities})
## replacing cities with digits

In [8]:
df_test = df_test.replace({'Source':dict_cities})
df_train = df_train.replace({'Source':dict_cities})

In [12]:
# replace destination
df_train = df_train.replace({'Destination':dict_cities})
df_test = df_test.replace({'Destination':dict_cities})

In [14]:
## replacing total stops with integers of 0,1,2,3,4 for respective number of stops , nan is taken as 0 stops
dict_stops={'non-stop':0,'2 stops':2,'1 stop':1,'3 stops':3,'4 stops':4 }
df_train['Total_Stops'].fillna(0, inplace=True)

In [15]:
#replace total stops in train set
df_train = df_train.replace({'Total_Stops':dict_stops})

In [16]:
df_test['Total_Stops'].fillna(0, inplace=True)

In [17]:
df_test = df_test.replace({'Total_Stops':dict_stops})

In [ ]:
## replacing additional info with digits
df_test['Additional_Info'].unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Long layover', 'Business class',
       'Change airports'], dtype=object)

In [20]:
dict_info={'No info':0,'No Info':0,'In-flight meal not included':1,'No check-in baggage included':2,'1 Short layover':3,'1 Long layover':4,'Change airports':5,'Business class':6,'Red-eye flight':7,'2 Long layover':8}

In [21]:
df_train = df_train.replace({'Additional_Info':dict_info})

In [22]:
df_test = df_test.replace({'Additional_Info':dict_info})

In [29]:
df_train['Flight hours']=df_train['Duration'].str.split('h').str[0]

In [31]:
df_test['Flight hours']=df_test['Duration'].str.split('h').str[0]

In [39]:
df_train['Airline'].unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [40]:
dict_airline={'IndiGo':0,'Air India':2,'Jet Airways':1,'SpiceJet':3,'Multiple carriers':4,'GoAir':5,'Vistara':6,'Air Asia':7,'Vistara Premium economy':8,'Jet Airways Business':9,'Multiple carriers Premium economy':10,'Trujet':11 }

In [41]:
df_train = df_train.replace({'Airline':dict_airline})
df_test = df_test.replace({'Airline':dict_airline})

In [48]:
y=df_train['Price']
train=df_train.copy()

In [57]:
train=train.drop(['Date_of_Journey','Dep_Time','Arrival_Time','Duration'],axis=1)

In [61]:
df_train['Flight hours']=df_train['Flight hours'].str.split('m').str[0]

In [63]:
train=train.drop(['Flight hours'],axis=1)

In [64]:
xgb = XGBRegressor(max_depth = 20,objective='reg:linear',n_estimators=500)
xgb.fit(train,y)

[05:39:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=20, min_child_weight=1, missing=None, n_estimators=500,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [65]:
xgb.score(train,y) 

0.9999999999987642